In [ ]:
import torch
from archisound import ArchiSound


In [ ]:
autoencoder = ArchiSound.from_pretrained('autoencoder1d-AT-v1')

x = torch.randn(1, 2, 2**18)    # [1, 2, 262144]
z = autoencoder.encode(x)       # [1, 32, 8192]
y = autoencoder.decode(z)       # [1, 2, 262144]